<a href="https://colab.research.google.com/github/JayThibs/gpt-experiments/blob/main/notebooks/gpt_2_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning GPT-2 on Alignment Texts Dataset

This notebook is meant for initial experimentation of fine-tuning on the alignment text dataset.

In [1]:
!nvidia-smi

Fri Jul  1 14:52:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installations

In [2]:
!pip install git+https://github.com/huggingface/transformers pytorch-lightning beautifulsoup4 datasets jsonlines ftfy lm_dataformat --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 585 kB 7.5 MB/s 
     |████████████████████████████████| 362 kB 23.3 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.3 MB/s 
     |████████████████████████████████| 596 kB 54.7 MB/s 
     |████████████████████████████████| 140 kB 55.4 MB/s 
     |████████████████████████████████| 419 kB 54.9 MB/s 
     |████████████████████████████████| 1.1 MB 50.9 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
     |████████████████████████████████| 127 kB 68.3 MB/s 
     |████████████████████████████████| 2.5 MB 41.5 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 52.6 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |█████

# Imports

In [3]:
import os
import re
import torch
import random
import jsonlines
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2TokenizerFast, AutoTokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
import ftfy
from lm_dataformat import Reader
pd.set_option('display.max_colwidth', None)

# Mounting Google Drive

Here we will mount our Google Drive so that we can grab data and save the HuggingFace scripts, and save the model once we've fine-tuned it.

In [4]:
# For saving the data locally
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd drive/MyDrive/data/ai-alignment-dataset/

/content/drive/MyDrive/data/ai-alignment-dataset


In [6]:
# !git clone https://github.com/JayThibs/gpt-experiments

# Data Preparation

## Clearning and Chunking Functions

Functions for preparing the data into chunks that can fit into GPT.

In [10]:
!python create_finetune_csv.py "alignment_texts.jsonl" "gpt" --normalize-with-ftfy --min-unique-tokens=10

reading/tokenizing files: 100% 34827/34827 [08:41<00:00, 66.83it/s]
enforce_min_unique_tokens: 100% 87765/87765 [00:03<00:00, 28657.78it/s]
87606
1000
dropped 389 tokens of trailing data


In [11]:
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [12]:
# import csv

# i = 0
# texts = []
# with jsonlines.open("alignment_texts.jsonl") as reader:
#     for line in reader:
#         text = line["text"]
#         texts.append(text)
#         if i > 3:
#             break
#         # try:
#         if text != "":
#             print(text)
#             print(len(text.split()))
#             encoding = tokenizer(text)
#             total_len = len(encoding.tokens())
#             tokens = encoding.tokens()
#             # print(tokens)
#             print(tokenizer.decode(encoding.input_ids))
#         # if total_len > 1024:
#         #     break
#         i += 1
#         # except:
#         #     pass

## Training Splits

In [18]:
alignment_texts = pd.read_csv("alignment_texts_87606.csv")

In [19]:
alignment_texts = list(alignment_texts)
alignment_texts[0]

'<|endoftext|> Braudel is probably the most impressive historian I have read. His quantitative estimates of premodern populations and crop yields are exactly the sort of foundation you\'d think any understanding of history would be based upon. Yet reading his magnum opus, it became steadily clearer as the books progressed that Braudel was missing some fairly fundamental economic concepts. I couldn\'t quite put my finger on *what* was missing until a section early in book 3:\n>... these deliberately simple tautologies make more sense to my mind than the so-called \'irrefutable\' pseudo-theorem of David Ricardo (1817), whose terms are well known: that the relations between two given countries depend on the "comparative costs" obtaining in them at the point of production\nBraudel, apparently, is not convinced by the principle of comparative advantage. What is his objection?\n> The division of labor on a world scale (or on world-economy-scale) cannot be described as a concerted agreement m

In [ ]:
train, val = train_test_split(musk_tweets, test_size=0.2)
test, val = train_test_split(val, test_size=0.5)

In [ ]:
print("Number of Train examples: " + str(len(train)))
print("Number of Val examples: " + str(len(val)))
print("Number of Test examples: " + str(len(test)))

Number of Train examples: 27148
Number of Val examples: 3394
Number of Test examples: 3393


In [ ]:
train_path = f'{directory}' + 'train.csv'
val_path = f'{directory}' + 'val.csv'
test_path = f'{directory}' + 'test.csv'

train.to_csv(train_path, index=False)
val.to_csv(val_path, index=False)
test.to_csv(test_path, index=False)

# Fine-Tuning GPT-2

If we're looking to fine-tune models which are found on the HuggingFace model hub, then it becomes much easier to fine-tune the models since HuggingFace provides us with scripts.

From the `transformers` repo:

> There are two sets of scripts provided. The first set leverages the Trainer API. The second set with no_trainer in the suffix uses a custom training loop and leverages the 🤗 Accelerate library. Both sets use the 🤗 Datasets library. You can easily customize them to your needs if you need extra processing on your datasets.

You can learn more about it here: https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling

We will be using the script that leveraged the Trainer API. We can download the script by running:

In [22]:
if not os.path.exists('/gpt-2/run_clm.py'):
    !wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/language-modeling/run_clm.py -P gpt-2/

--2022-07-01 16:02:39--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25025 (24K) [text/plain]
Saving to: ‘gpt-2/run_clm.py’

run_clm.py          100%[===================>]  24.44K  --.-KB/s    in 0.004s  

2022-07-01 16:02:39 (6.13 MB/s) - ‘gpt-2/run_clm.py’ saved [25025/25025]



# Train

In [23]:
!python gpt-2/run_clm.py \
    --model_name_or_path gpt2 \
    --train_file alignment_texts_87606.csv \
    --do_train \
    --fp16 \
    --overwrite_cache \
    --per_device_train_batch_size=2 \
    --output_dir gpt-2/tmp/alignment-texts-clm \
    --overwrite_output_dir

07/01/2022 16:02:47 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/01/2022 16:02:47 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private

# Let's use the model!

In [ ]:
OUTPUT_DIR = "gpt-2/tmp/elon-test-clm"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
                                        
def generate(input_str, length=250, n=5):
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text

def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

generated_text = generate("Just dropping some")
print(generated_text)

Just dropping some of my old stuff in the trunk. It’s been a while since I last used a car.      _AA_Carmack   Yeah, I think we should do something about it.  We have a long way to go. Will be interesting to see what happens to those who don’t support this.   I think it will be great. It will be a lot more than a mere cameo. _AA_Carmack  Yes, it’s a great game. I think we should do something about it.   _Station _Ryan _AA_Carmack       _Station _AA_Carmack  I’m not a big fan of the Tesla Autopilot software, but I do like the idea of having a car capable of recognizing pedestrians and cyclists. It’s a great idea, especially with the high speed autotracing. _Station I love you         _Padival         _Station  Yes, it will have a lot of new features coming to the Tesla Model S, including the ability to drive from the garage to the


In [ ]:
generated_text = generate("Just dropping some")
print(generated_text)

Just dropping some of our own resources into the ocean                   Yeah, we have to make sure we have a good product. That will be a priority.  We will make a new version of Falcon Heavy for free.  _AA_Carmack      _Sword _Sword _Sword    _AA_Carmack _Sword          We’ll try to get that done, but I think we’ll be better off with a more advanced, reusable, reusable rocket booster.        Yeah, I love it :)       I love the idea of having a Tesla in the car. It’s awesome.  _Ryan    Yeah, that's what we should do   Yeah, we will make the Model S a lot faster, but we have a lot of work to do to get it right, as we did in the beginning. We’ve had a lot of setbacks._Gardi  _Ryan Yeah, that's exactly right _AA_Carmack I’m just trying to be as polite


# Compressing the Model

Let's save the model as a `tar.gz` file so that we can save it in Google Drive.

In [ ]:
!tar -czf gpt-2-elon-tweets.tar.gz gpt-2/tuned-models/